In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_excel("DataCore.xlsx")

In [ ]:
df

In [ ]:
df = df.rename(columns={'INF': 'INFLATION'})
df.drop(['ROE', 'NIM','CUS'], axis=1, inplace=True)

In [ ]:
df

In [ ]:
df.describe(include='all')

In [ ]:
bien_doc_lap = ['DEP','CASH','LGAP','NPL','SIZE', 'CR3','VCSH', 'RRTD','GDP', 'INFLATION']
bien_phu_thuoc = ['ROA']
danh_sach_bien = ['ROA','DEP','CASH','LGAP','NPL','SIZE', 'CR3','VCSH', 'RRTD','GDP', 'INFLATION']

In [ ]:
#phân tích ma trận tự tương quan của các biến độc lập
corr_matrix = df[bien_doc_lap].corr()
corr_matrix

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm', center=0)
plt.title('Bảng kết quả phân tích tự tương quan của các biến độc lập trong mô hình')

In [ ]:
y_bien = 'ROA'
n = len(bien_doc_lap)

# Tạo Figure: n hàng, 3 cột
fig, axes = plt.subplots(nrows=n, ncols=3, figsize=(18, n * 5))

if n == 1:
    axes = axes.reshape(1, -1)

for i, col in enumerate(bien_doc_lap):
    # Tính toán các chỉ số
    s_val = df[col].skew()
    max_val = df[col].max()

    # 1. Boxplot dọc (Cột 0) - Hiển thị râu đến Max tuyệt đối
    sns.boxplot(y=df[col], ax=axes[i, 0], color='skyblue', width=0.4, whis=(0, 100))
    axes[i, 0].set_title(f'Boxplot của {col}\n(Max: {max_val:.2f})')
    axes[i, 0].set_ylabel('Giá trị')

    # 2. Histogram thực thụ (Cột 1) - Thể hiện độ lệch
    sns.histplot(df[col], kde=True, ax=axes[i, 1], color='green',
                 bins='auto',           # Tự động tính số cột tối ưu
                 edgecolor='white',    # Thêm viền trắng giữa các cột cho rõ ràng
                 line_kws={'linewidth': 3}) # Làm đậm đường cong KDE

    axes[i, 1].set_title(f'Histogram & Độ lệch: {s_val:.2f}')
    axes[i, 1].set_xlabel(col)
    axes[i, 1].set_ylabel('Tần suất')
    # Đường Median (Trung vị) để đối chiếu độ lệch
    axes[i, 1].axvline(df[col].median(), color='blue', linestyle='--', label='Median')
    axes[i, 1].legend()

    # 3. Regplot (Cột 2) - Mối quan hệ với ROA
    sns.regplot(data=df, x=col, y=y_bien, ax=axes[i, 2],
                scatter_kws={'alpha':0.4, 's':25},
                line_kws={'color':'red'})
    axes[i, 2].set_title(f'Hồi quy: {col} vs {y_bien}')
    axes[i, 2].set_xlabel(col)
    axes[i, 2].set_ylabel(y_bien)
plt.tight_layout()


In [ ]:
# thực hiện kiểm định đa cộng tuyến
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.api import add_constant

def check_vif_final(df, bien_doc_lap):
    # 1. Chuẩn bị dữ liệu
    X = df[bien_doc_lap].dropna()
    X_with_const = add_constant(X)

    # 2. Tính VIF và làm tròn
    vif_data = pd.DataFrame()
    vif_data["Biến độc lập"] = X_with_const.columns
    vif_data["VIF"] = [variance_inflation_factor(X_with_const.values, i)
                        for i in range(len(X_with_const.columns))]

    # 3. Lọc bỏ 'const', sắp xếp và làm tròn 2 chữ số
    vif_result = vif_data[vif_data["Biến độc lập"] != "const"].copy()
    vif_result["VIF"] = vif_result["VIF"].round(2)
    vif_result = vif_result.sort_values(by="VIF", ascending=False).reset_index(drop=True)

    # 4. Tính trung bình và làm tròn
    vif_mean = round(vif_result["VIF"].mean(), 2)

    # 5. Thêm hàng trung bình
    mean_row = pd.DataFrame([{"Biến độc lập": "--- TRUNG BÌNH ---", "VIF": vif_mean}])
    vif_final = pd.concat([vif_result, mean_row], ignore_index=True)

    return vif_final


vif_table = check_vif_final(df, bien_doc_lap)
print(vif_table)

In [ ]:
# Thực hiện các kiểm định để lựa chọn mô hình hồi quy phù hợp trong các mô hình: Pooled OLS, FEM (Fixed Effects), REM (Random Effects)
## Bước 1: Kiểm định F-test (So sánh Pooled OLS và Fixed Effects - Kiểm định này xem xét liệu có sự khác biệt đặc thù giữa các đơn vị quan sát hay không.)
## Giả thuyết Ho: Mô hình Pooled OLS là phù hợp (không có đặc điểm riêng biệt giữa các đơn vị).
### Nếu p-value < 0.05: Bác bỏ Ho:, chọn mô hình Fixed Effects (FE).
### Nếu p-value > 0.05: Chọn Pooled OLS
from linearmodels import PooledOLS, PanelOLS, RandomEffects
import statsmodels.api as sm

# 1. Chuẩn bị dữ liệu
df_panel = df.set_index(['BANK', 'YEAR'])
exog = sm.add_constant(df_panel[bien_doc_lap])
endog = df_panel['ROA']

# 2. Chạy các mô hình
mod_pooled = PooledOLS(endog, exog).fit()
mod_fe = PanelOLS(endog, exog, entity_effects=True).fit()
mod_re = RandomEffects(endog, exog).fit()

# 3. In kết quả để so sánh p-value của các kiểm định đính kèm
print(mod_pooled)

In [ ]:
print(mod_fe) # Xem F-test for individual effects

In [ ]:
print(mod_re)

In [ ]:
# ==============================================================================
# KIỂM ĐỊNH BREUSCH-PAGAN LAGRANGE MULTIPLIER (LM TEST)
# Mục đích: Lựa chọn giữa mô hình Pooled OLS và Random Effects (REM)
# Giả thuyết H0: Phương sai của sai số riêng biệt bằng 0 (Pooled OLS phù hợp).
# Giả thuyết H1: Phương sai của sai số riêng biệt khác 0 (REM phù hợp).
# ==============================================================================
from scipy.stats import chi2

def breusch_pagan_lm_test(residuals, group_variable):
    # Lấy n (số lượng ngân hàng) và T (số năm trung bình)
    n = group_variable.nunique()
    # Tự động tính T dựa trên số lượng mẫu
    T = len(residuals) / n

    # Tính tổng bình phương sai số của Pooled OLS
    sum_sq_residuals = np.sum(residuals**2)

    # Gom nhóm sai số theo Ngân hàng và tính tổng của từng nhóm, sau đó bình phương
    # Lưu ý: residuals từ linearmodels có index là (BANK, YEAR), ta group theo level 0 (BANK)
    grouped_sum = residuals.groupby(level=0).sum()
    sum_grouped_sq = np.sum(grouped_sum**2)

    # Tính thống kê LM
    lm_stat = (n * T / (2 * (T - 1))) * ((sum_grouped_sq / sum_sq_residuals) - 1)**2

    # Tính P-value (với bậc tự do = 1)
    p_value = 1 - chi2.cdf(lm_stat, df=1)

    print("\n--- KIỂM ĐỊNH BREUSCH-PAGAN LM (Pooled OLS vs REM) ---")
    print(f"LM Statistic: {lm_stat:.4f}")
    print(f"P-value:      {p_value:.4f}")

    if p_value < 0.05:
        print("=> Kết luận: Bác bỏ H0. Có hiệu ứng ngẫu nhiên. Mô hình REM phù hợp hơn Pooled OLS.")
    else:
        print("=> Kết luận: Chấp nhận H0. Không có hiệu ứng ngẫu nhiên. Mô hình Pooled OLS phù hợp hơn.")

# Gọi hàm kiểm định
# Lưu ý: mod_pooled.resids là phần dư của mô hình Pooled OLS đã chạy ở trên
breusch_pagan_lm_test(mod_pooled.resids, df['BANK'])

In [ ]:
# Kiểm định Hausman (So sánh Fixed Effects và Random Effects)
## kiểm tra xem sai số đặc thù có tương quan với các biến độc lập hay không
## Giả thuyết Ho: Mô hình Random Effects (RE) là phù hợp.
### Nếu p-value < 0.05: Bác bỏ Ho, chọn mô hình Fixed Effects (FE).
### Nếu p-value > 0.05: Chọn mô hình Random Effects (RE).
import scipy.stats as stats

def hausman(fe, re):
    b = fe.params
    B = re.params
    v_b = fe.cov
    v_B = re.cov
    df = b.shape[0]
    chi2 = np.dot((b - B).T, np.linalg.inv(v_b - v_B).dot(b - B))
    p_val = stats.chi2.sf(chi2, df)
    return chi2, p_val

chi2, p_val = hausman(mod_fe, mod_re)
print(f"Hausman Test: Chi2 = {chi2:.2f}, p-value = {p_val:.4f}")

In [ ]:
# ==============================================================================
# TRỰC QUAN HÓA KHUYẾT TẬT CỦA MÔ HÌNH FEM
# ==============================================================================
import matplotlib.pyplot as plt
import seaborn as sns

def visualize_model_defects(model_fe):
    """
    Hàm vẽ biểu đồ chẩn đoán khuyết tật mô hình FEM
    """
    # 1. Chuẩn bị dữ liệu
    # Lấy phần dư (residuals) và giá trị dự báo (fitted values)
    # Lưu ý: Với linearmodels, predict() trả về DataFrame, resids là Series
    residuals = model_fe.resids
    fitted_values = model_fe.predict()

    # Tạo DataFrame tạm để xử lý
    plot_data = pd.DataFrame({
        'Residuals': residuals.iloc[:, 0] if isinstance(residuals, pd.DataFrame) else residuals,
        'Fitted_Values': fitted_values.iloc[:, 0]
    })

    # Tạo biến trễ cho phần dư (Lagged Residuals) để check tự tương quan
    # Index của linearmodels là (BANK, YEAR), ta group theo level 0 (BANK) để lag đúng đinh dạng bảng
    plot_data['Lagged_Residuals'] = plot_data.groupby(level=0)['Residuals'].shift(1)

    # 2. Thiết lập khung hình
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))


    # --- BIỂU ĐỒ 1: KIỂM TRA PHƯƠNG SAI SAI SỐ THAY ĐỔI ---
    # Vẽ Scatter plot: Trục hoành là Giá trị dự báo, Trục tung là Phần dư
    sns.scatterplot(x='Fitted_Values', y='Residuals', data=plot_data, ax=axes[0], color='blue', alpha=0.6)
    # Vẽ đường trung bình 0
    axes[0].axhline(0, color='red', linestyle='--', linewidth=2)

    axes[0].set_title('1. Kiểm tra Phương sai thay đổi\n(Residuals vs. Fitted Values)', fontweight='bold')
    axes[0].set_xlabel('Giá trị dự báo (Fitted Values)')
    axes[0].set_ylabel('Phần dư (Residuals)')

    # Ghi chú dấu hiệu nhận biết
    axes[0].text(0.05, 0.90, 'Dấu hiệu: Các điểm loe ra hình cái phễu\nhoặc cánh bướm -> Có Phương sai thay đổi',
                 transform=axes[0].transAxes, fontsize=10, bbox=dict(facecolor='#ffcccc', alpha=0.8))

    # --- BIỂU ĐỒ 2: KIỂM TRA TỰ TƯƠNG QUAN ---
    # Vẽ Scatter plot: Trục hoành là Phần dư kỳ trước, Trục tung là Phần dư kỳ này
    sns.scatterplot(x='Lagged_Residuals', y='Residuals', data=plot_data, ax=axes[1], color='green', alpha=0.6)

    # Vẽ đường hồi quy tuyến tính để thấy rõ xu hướng
    sns.regplot(x='Lagged_Residuals', y='Residuals', data=plot_data, ax=axes[1],
                scatter=False, color='red', line_kws={'linestyle':'--'})

    axes[1].set_title('2. Kiểm tra Tự tương quan\n(Residuals vs. Lagged Residuals)', fontweight='bold')
    axes[1].set_xlabel('Phần dư kỳ trước (t-1)')
    axes[1].set_ylabel('Phần dư kỳ này (t)')

    # Ghi chú dấu hiệu nhận biết
    axes[1].text(0.05, 0.90, 'Dấu hiệu: các điểm bám theo đường chéo\n(dốc lên/xuống) -> Có Tự tương quan',
                 transform=axes[1].transAxes, fontsize=10, bbox=dict(facecolor='#ccffcc', alpha=0.8))

# --- GỌI HÀM ---
# Chạy hàm này ngay sau khi có biến 'mod_fe'
visualize_model_defects(mod_fe)

In [ ]:
# Kiểm định các khuyết tật (Diagnostics): Tự tương quan (Autocorrelation) và Phương sai sai số thay đổi (Heteroskedasticity)
# 1. Kiểm định Phương sai sai số thay đổi (Heteroskedasticity)
import statsmodels.stats.api as sms

# Giả sử 'mod_fe' là kết quả từ PanelOLS(endog, exog, entity_effects=True).fit()
def check_heteroskedasticity(model):
    # Lấy phần dư
    residuals = model.resids

    # Kiểm định Breusch-Pagan
    # Cần dùng ma trận biến độc lập từ mô hình
    exog = model.model.exog.values2d
    names = ['Lagrange multiplier statistic', 'p-value', 'f-value', 'f p-value']
    test = sms.het_breuschpagan(residuals, exog)

    print("--- Kiểm định Phương sai sai số thay đổi (Breusch-Pagan) ---")
    print(dict(zip(names, test)))

    if test[1] < 0.05:
        print("Kết quả: Có hiện tượng Phương sai sai số thay đổi (p < 0.05)")
    else:
        print("Kết quả: Không có hiện tượng Phương sai sai số thay đổi")

check_heteroskedasticity(mod_fe)

In [ ]:
import statsmodels.formula.api as smf
import statsmodels.api as sm

def wooldridge_autocorrelation_test(df, y_col, x_cols, entity_col, time_col):
    """
    Thực hiện kiểm định Wooldridge cho tự tương quan trong dữ liệu bảng.
    H0: Không có tự tương quan bậc 1 (First-order autocorrelation).
    H1: Có tự tương quan.
    """
    print(f"\n--- KIỂM ĐỊNH WOOLDRIDGE CHO TỰ TƯƠNG QUAN ---")

    # 1. Sắp xếp dữ liệu và lấy sai phân bậc 1 (First Difference)
    df_sorted = df.sort_values([entity_col, time_col])
    # Gom nhóm theo Entity để tính sai phân đúng
    df_diff = df_sorted.groupby(entity_col)[x_cols + [y_col]].diff().dropna()

    # 2. Hồi quy OLS trên dữ liệu sai phân (Không có hằng số - No Constant)
    # Công thức: D.Y ~ D.X - 1
    # Dùng sm.OLS thay vì formula để dễ xử lý list biến
    model_diff = sm.OLS(df_diff[y_col], df_diff[x_cols], hasconst=False).fit()

    # 3. Lấy phần dư (Residuals)
    residuals = model_diff.resid

    # 4. Tạo biến trễ của phần dư (Lagged Residuals)
    # Cần map phần dư quay lại dataframe có thông tin Entity để lag chính xác
    df_resid = pd.DataFrame({'resid': residuals}, index=df_diff.index)
    # Join lại với df gốc (đã sort) để lấy cột Entity
    df_resid = df_resid.join(df_sorted[[entity_col]])

    # Tạo lag
    df_resid['resid_lag'] = df_resid.groupby(entity_col)['resid'].shift(1)
    df_resid = df_resid.dropna()

    # 5. Hồi quy phần dư theo phần dư trễ
    # Công thức: resid ~ resid_lag
    model_final = smf.ols('resid ~ resid_lag', data=df_resid).fit()

    # 6. Kiểm định giả thuyết H0: Hệ số của resid_lag = -0.5
    # Nếu p-value < 0.05: Bác bỏ H0 -> Có tự tương quan
    # Tự tính F-test hoặc t-test cho giả thuyết Coeff = -0.5
    hypotheses = 'resid_lag = -0.5'
    f_test = model_final.f_test(hypotheses)

    f_stat = f_test.fvalue.item() if hasattr(f_test.fvalue, 'item') else f_test.fvalue
    p_val = f_test.pvalue.item() if hasattr(f_test.pvalue, 'item') else f_test.pvalue

    print(f"F-Statistic: {f_stat:.4f}")
    print(f"P-value:     {p_val:.4f}")

    if p_val < 0.05:
        print("=> Kết luận: Bác bỏ H0. CÓ hiện tượng tự tương quan (Serial Correlation).")
    else:
        print("=> Kết luận: Chấp nhận H0. KHÔNG có hiện tượng tự tương quan.")

    return p_val

wooldridge_autocorrelation_test(df, 'ROA', bien_doc_lap, 'BANK', 'YEAR')

In [ ]:
## Qua các kết quả kiểm định nêu trên cho thấy mô hình hồi quy theo phương pháp bình phương bé nhất Pooled OLS, FEM là không phù hợp, vì mô hình vi phạm các giả định hồi quy như tự tương quan và phương sai thay đổi. Khi đó sẽ dẫn tới kết quả hồi quy không còn chính xác. Để khắc phục những vi phạm trên bài viết tiếp tục dùng phương pháp bình phương bé nhất tổng quát (GLS) để khắc phục các hiện tượng trên.

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

# 1. Định nghĩa công thức
formula = "ROA ~ DEP + CASH + LGAP + NPL + SIZE + CR3 + VCSH + RRTD + GDP + INFLATION"

# 2. Khởi tạo mô hình GEE
model = smf.gee(formula=formula,
                groups=df["BANK"],
                data=df,
                cov_struct=sm.cov_struct.Exchangeable(),
                family=sm.families.Gaussian())

# 3. Fit mô hình với cov_type='naive'
# 'naive' sẽ tính toán sai số chuẩn dựa trên giả định mô hình (giống xtgls mặc định)
# giúp P-value của INFLATION và SIZE thu nhỏ lại sát với Stata.
results = model.fit(cov_type='naive')

# 4. Hiển thị bảng kết quả
print("--- KẾT QUẢ MÔ HÌNH GLS (GEE) ---")
print(results.summary())

# 5. Kiểm định Wald tổng thể
print("\n--- KIỂM ĐỊNH WALD TỔNG THỂ ---")
print(results.wald_test_terms())

In [ ]:
# 1. Lấy giá trị dự báo từ mô hình
y_pred = results.predict(df)

# 2. Lấy giá trị thực tế (ROA)
y_actual = df['ROA']

# 3. Tính hệ số tương quan bình phương (tương đương R-squared)
correlation_matrix = np.corrcoef(y_actual, y_pred)
correlation_xy = correlation_matrix[0,1]
r_squared = correlation_xy**2

print(f"Hệ số xác định R-squared của mô hình là: {r_squared:.4f}")
print(f"Mô hình GLS giải thích được: {r_squared*100:.2f}% sự biến động của ROA")

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# 1. Huấn luyện mô hình RF
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(df[bien_doc_lap], df['ROA'])

# 2. Lấy tầm quan trọng của biến
importances = pd.Series(rf.feature_importances_, index=bien_doc_lap).sort_values(ascending=False)
print("Tầm quan trọng của các biến theo Random Forest:")
print(importances)

In [ ]:
# ==============================================================================
# MÔ HÌNH DỰ BÁO KẾT HỢP
# ==============================================================================
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

# 1. Chia dữ liệu TRƯỚC KHI TRAIN
# shuffle=False học quá khứ -> đoán tương lai
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)

print(f"Số quan sát huấn luyện: {len(train_df)}")
print(f"Số quan sát kiểm tra: {len(test_df)}")

# --- BƯỚC 1: CHẠY LẠI GLS CHỈ TRÊN TẬP TRAIN ---
model_train = smf.gee(formula=formula,
                groups=train_df["BANK"],
                data=train_df,
                cov_struct=sm.cov_struct.Exchangeable(),
                family=sm.families.Gaussian())

# Fit mô hình (dùng naive để khớp logic Stata)
results_train = model_train.fit(cov_type='naive')

# Dự báo GLS cho tập Train và tập Test
train_df['GLS_Pred'] = results_train.predict(train_df)
test_df['GLS_Pred'] = results_train.predict(test_df) # dự báo cho tập test

# Tính sai số (Residuals) trên tập Train
train_df['Residuals'] = train_df['ROA'] - train_df['GLS_Pred']

# --- BƯỚC 2: DẠY RANDOM FOREST HỌC SAI SỐ (CHỈ TRÊN TẬP TRAIN) ---
rf_residual = RandomForestRegressor(n_estimators=200, max_depth=5, random_state=42)
rf_residual.fit(train_df[bien_doc_lap], train_df['Residuals'])

# --- BƯỚC 3: DỰ BÁO KẾT HỢP TRÊN TẬP TEST ---
# 1. RF dự báo phần sai số cho tập Test
test_df['RF_Residual_Pred'] = rf_residual.predict(test_df[bien_doc_lap])

# 2. Cộng gộp: Hybrid = GLS (Tuyến tính) + RF (Phi tuyến)
test_df['Hybrid_Pred'] = test_df['GLS_Pred'] + test_df['RF_Residual_Pred']

# --- BƯỚC 4: ĐÁNH GIÁ HIỆU QUẢ ---
def evaluate_model(y_true, y_pred, name):
    r2 = r2_score(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    print(f"--- {name} ---")
    print(f"R-squared: {r2:.4f} ({r2*100:.2f}%)")
    print(f"RMSE: {rmse:.5f}")

print("\n=== KẾT QUẢ SO SÁNH TRÊN TẬP DỮ LIỆU KIỂM TRA (TEST SET) ===")
evaluate_model(test_df['ROA'], test_df['GLS_Pred'], "GLS Truyền thống")
evaluate_model(test_df['ROA'], test_df['Hybrid_Pred'], "Mô hình kết hợp (GLS + RF)")